# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

In [47]:
import pandas as pd
url = 'https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv'
df = pd.read_csv(url)

In [48]:
# Cleaning the titles
def clean_titles(data_frame):
    columns = list(data_frame.columns)
    data_frame.columns = [i.lower().replace(' ','_').strip().replace('employmentstatus', 'employment_status') for i in columns]
    return data_frame
clean_titles(df)
# Removing duplicated index
df.drop(labels='unnamed:_0', axis =1, inplace=True)

In [ ]:
# Removing duplicates
df.drop_duplicates(inplace=True)

1. Create a new DataFrame that only includes customers who:
   - have a **low total_claim_amount** (e.g., below $1,000),
   - have a response "Yes" to the last marketing campaign.

In [82]:
condition = (df['total_claim_amount'] < 1000) & (df['response'] == 'Yes')
df_filtered = df[condition]

In [89]:
df['monthly_premium_auto'].dtype

dtype('int64')

2. Using the original Dataframe, analyze:
   - the average `monthly_premium` and/or customer lifetime value by `policy_type` and `gender` for customers who responded "Yes", and
   - compare these insights to `total_claim_amount` patterns, and discuss which segments appear most profitable or low-risk for the company.

In [103]:
condition = df['response'] == 'Yes'
(df[condition].pivot_table(index=['policy_type','gender'], values= ['monthly_premium_auto', 'customer_lifetime_value', 'total_claim_amount'])).round(2)

customer_lifetime_value  monthly_premium_auto  \
policy_type    gender                                                  
Corporate Auto F                       7712.63                 94.30   
               M                       7928.36                 92.13   
Personal Auto  F                       8339.10                 98.96   
               M                       7448.38                 91.09   
Special Auto   F                       7691.58                 92.31   
               M                       8247.09                 86.34   

                       total_claim_amount  
policy_type    gender                      
Corporate Auto F                   433.74  
               M                   407.83  
Personal Auto  F                   452.76  
               M                   457.01  
Special Auto   F                   453.28  
               M                   429.53

In [104]:
# Data for the whole df, ignoring the response condition
(df.pivot_table(index=['policy_type','gender'], values= ['monthly_premium_auto', 'customer_lifetime_value', 'total_claim_amount'])).round(2)

customer_lifetime_value  monthly_premium_auto  \
policy_type    gender                                                  
Corporate Auto F                       7967.70                 91.46   
               M                       7730.98                 94.77   
Personal Auto  F                       8077.14                 93.19   
               M                       7974.13                 93.29   
Special Auto   F                       8460.40                 93.56   
               M                       9010.60                 93.20   

                       total_claim_amount  
policy_type    gender                      
Corporate Auto F                   398.21  
               M                   462.37  
Personal Auto  F                   413.34  
               M                   459.69  
Special Auto   F                   458.14  
               M                   420.36

In [ ]:
# Analysis based on OVERALL customer lifetime value:
# Female customers appear to be more profitable (albeit marginally) for the company when hiring 'Corporate' or 'Personal'
# policies, while male customers are a lot more profitable for the 'Special' policy.

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

In [130]:
df_state = df.groupby('state').agg({'state':'count'})
condition = df_state['state'] > 500
df_state[condition]

,state
state,
Arizona,1934
California,3548
Nevada,992
Oregon,2897
Washington,888


4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

In [145]:
#for comparing between differenr education levels on each gender
# for a better comparison between genders
print('Comparison between education levels\n------------------------------------')
display(df.groupby(['gender', 'education']).agg({'customer_lifetime_value':['max','min','median','mean']}).round(2))

# for a better comparison between genders
print('Comparison between genders\n----------------------------')
display(df.groupby(['gender', 'education']).agg({'customer_lifetime_value':['max','min','median','mean']}).round(2).unstack())


Comparison between education levels
------------------------------------


customer_lifetime_value                           
                                                max      min   median     mean
gender education                                                              
F      Bachelor                            73225.96  1904.00  5632.61  7879.74
       College                             61850.19  1898.68  5623.61  7749.67
       Doctor                              44856.11  2395.57  5332.46  7329.62
       High School or Below                55277.45  2144.92  6035.09  8665.64
       Master                              51016.07  2417.78  5729.86  8157.05
M      Bachelor                            67907.27  1898.01  5548.03  7710.90
       College                             61134.68  1918.12  6005.85  8041.39
       Doctor                              32677.34  2267.60  5577.67  7415.33
       High School or Below                83325.38  1940.98  6286.73  8147.12
       Master                              50568.26  2272.31  5579.10  8168.83

Comparison between genders
----------------------------


customer_lifetime_value                                           \
                              max                                            
education                Bachelor   College    Doctor High School or Below   
gender                                                                       
F                        73225.96  61850.19  44856.11             55277.45   
M                        67907.27  61134.68  32677.34             83325.38   

                                                                              \
                         min                                                   
education    Master Bachelor  College   Doctor High School or Below   Master   
gender                                                                         
F          51016.07  1904.00  1898.68  2395.57              2144.92  2417.78   
M          50568.26  1898.01  1918.12  2267.60              1940.98  2272.31   

                                                                             \
            median                                                     mean   
education Bachelor  College   Doctor High School or Below   Master Bachelor   
gender                                                                        
F          5632.61  5623.61  5332.46              6035.09  5729.86  7879.74   
M          5548.03  6005.85  5577.67              6286.73  5579.10  7710.90   

                                                           
                                                           
education  College   Doctor High School or Below   Master  
gender                                                     
F          7749.67  7329.62              8665.64  8157.05  
M          8041.39  7415.33              8147.12  8168.83

In [ ]:
# In all cases the median is always closer to the min value, which indicates that regardless of education and gender, there
# are always a few users that have very high customer lifetime values.

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [ ]:
# your code goes here